# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [4]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [6]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = [64, 32] #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [7]:
# defining variables 

img_idx= [x for x in range(0,30,2)]
img_width, img_height= 84, 84
channels= 3

In [8]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    while True:

        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches

        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx), img_width, img_height,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder

                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    crop_image= image[10:, 10: ,:]
                    resize_image= imresize(crop_image, (img_width, img_height, channels))

                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    batch_data[folder,idx,:,:,0] = (resize_image[:,:, 0])//255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (resize_image[:,:, 1])//255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (resize_image[:,:, 2])//255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches

        if (len(t)%batch_size)!= 0: 
            
            batch_size= len(t)%batch_size
            batch_data = np.zeros((batch_size,len(img_idx), img_width, img_height,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch_size)].split(';')[0]) # read all the images in the folder

                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    crop_image= image[10:, 10: ,:]
                    resize_image= imresize(crop_image, (img_width, img_height, channels))

                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    batch_data[folder,idx,:,:,0] = (resize_image[:,:, 0])//255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (resize_image[:,:, 1])//255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (resize_image[:,:, 2])//255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [58]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

### Convolutional 3D 

### Basic Convolutional Model with Batch Normalization and Without Dropout

In [78]:
# Conv3D()
input_shape= (len(img_idx), img_width, img_height, channels)
model2 = Sequential()

model2.add(Conv3D(16, (3,3,3), strides=(1,1,1),padding= 'same', 
                 input_shape= input_shape, name="conv1"))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling3D(pool_size= (2,2,1), strides= (2,2,1)))

model2.add(Conv3D(32, (3,3,3), strides=(1,1,1),padding= 'same', 
                 input_shape= input_shape, name= "conv2"))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling3D(pool_size= (2,2,1), strides= (2,2,1)))
          
model2.add(Conv3D(64, (3,3,3), strides= (1,1,1), padding= 'same', 
                 input_shape= input_shape, name= "conv3"))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling3D(pool_size= (2,2,1), strides= (2,2,1)))
          
model2.add(Conv3D(128, (3,3,3), strides= (1,1,1),padding='same', 
                 input_shape= input_shape, name= "conv4"))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling3D(pool_size=(2,2,1), strides= (2,2,1), padding= 'same'))

model2.add(Flatten())

model2.add(Dense(512, activation= 'relu'))
model2.add(BatchNormalization())
model2.add(Dense(5, activation= 'softmax'))

Exception ignored in: <function MediaFileUpload.__del__ at 0x7fc06bcffe60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/http.py", line 587, in __del__
    self._fd.close()
AttributeError: 'MediaFileUpload' object has no attribute '_fd'


In [79]:
optimiser = optimizers.Adam(lr= 0.001) #write your optimizer
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 15, 84, 84, 16)    1312      
_________________________________________________________________
activation_8 (Activation)    (None, 15, 84, 84, 16)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 84, 84, 16)    64        
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 7, 42, 84, 16)     0         
_________________________________________________________________
conv2 (Conv3D)               (None, 7, 42, 84, 32)     13856     
_________________________________________________________________
activation_9 (Activation)    (None, 7, 42, 84, 32)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 42, 84, 32)    

#### Batch Size - 64
#### Epochs - 50

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [99]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


In [100]:
train_generator = generator(train_path, train_doc, batch_size[0])
val_generator = generator(val_path, val_doc, batch_size[0])

In [101]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 2, 
                       verbose = 0, mode = "auto", epsilon = 1e-04, 
                       cooldown = 0,min_lr = 0) # write the REducelronplateau code here

ES= EarlyStopping(monitor= 'val_loss', min_delta= 0, patience= 10, 
                  verbose= 0, mode= 'auto', baseline= None)

callbacks_list = [checkpoint, LR, ES]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [102]:
if (num_train_sequences%batch_size[0]) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size[0])
else:
    steps_per_epoch = (num_train_sequences//batch_size[0]) + 1

if (num_val_sequences%batch_size[0]) == 0:
    validation_steps = int(num_val_sequences/batch_size[0])
else:
    validation_steps = (num_val_sequences//batch_size[0]) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [103]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


Source path =  Project_data/train ; batch size = 64
Epoch 1/50
 9/11 [=======================>......] - ETA: 6s - loss: 0.1770 - categorical_accuracy: 0.9531

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


11/11 [==============================] - 38s 4s/step - loss: 0.3203 - categorical_accuracy: 0.9276 - val_loss: 2.2187 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 2.21869, saving model to model_init_2021-05-0914_19_05.226677/model-00001-0.32029-0.92760-2.21869-0.21000.h5
Epoch 2/50
11/11 [==============================] - 18s 2s/step - loss: 0.1970 - categorical_accuracy: 0.9447 - val_loss: 2.0580 - val_categorical_accuracy: 0.1562

Epoch 00002: val_loss improved from 2.21869 to 2.05796, saving model to model_init_2021-05-0914_19_05.226677/model-00002-0.19701-0.94466-2.05796-0.15625.h5
Epoch 3/50
11/11 [==============================] - 16s 2s/step - loss: 0.1791 - categorical_accuracy: 0.9526 - val_loss: 2.0888 - val_categorical_accuracy: 0.2321

Epoch 00003: val_loss did not improve from 2.05796
Epoch 4/50
11/11 [==============================] - 14s 1s/step - loss: 0.4903 - categorical_accuracy: 0.8884 - val_loss: 2.1443 - val_categorical_accuracy: 